<script src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-MML-AM_CHTML"></script>

<script src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-MML-AM_CHTML"></script>

In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt

In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score

In [4]:
from sklearn.impute import SimpleImputer

In [5]:
from sklearn.compose import make_column_selector,make_column_transformer,ColumnTransformer

In [6]:
from sklearn.svm import SVR,SVC

In [7]:
from sklearn.ensemble import RandomForestRegressor

In [9]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler

Support Vector Machines (SVMs) are a versatile class of supervised learning algorithms used for both classification (SVC - Support Vector Classification) and regression (SVR - Support Vector Regression). The performance of SVM models heavily depends on the choice of the **kernel function**, which transforms the input data into a higher-dimensional space to make it separable or to fit a more complex function.

### **List of Kernel Types for SVM (SVR & SVC):**

#### **1. Linear Kernel**
   - Formula: $ K(x_i, x_j) = x_i^T x_j $
   - Used when data is linearly separable.
   - Fast and works well for high-dimensional data.
   - **Use Case**: Text classification, linear regression.

#### **2. Polynomial Kernel**
   - Formula: $K(x_i, x_j) = (\gamma x_i^T x_j + r)^d $
   - Parameters:
     - \( $\gamma$ ) (gamma): Controls influence of each training example.
     - \( $d$ ) (degree): Degree of the polynomial.
     - \( $r$ ) (coef0): Independent term.
   - **Use Case**: Moderate non-linearity, image processing.

#### **3. Radial Basis Function (RBF) / Gaussian Kernel**
   - Formula: $ K(x_i, x_j) = \exp(-\gamma \|x_i - x_j\|^2) $
   - Most commonly used kernel.
   - Parameter:
     - ( $\gamma $): Controls the "spread" of the kernel (inverse of variance).
   - **Use Case**: Highly non-linear problems, default choice for many tasks.

#### **4. Sigmoid Kernel (Hyperbolic Tangent Kernel)**
   - Formula: $K(x_i, x_j) = \tanh(\gamma x_i^T x_j + r) $
   - Similar to neural network activation functions.
   - Parameters:
     - ( $\gamma $): Scaling factor.
     - ( $r$): Bias term.
   - **Use Case**: Neural network-like models, but less commonly used than RBF.

#### **5. Custom Kernels**
   - Users can define their own kernel functions as long as they satisfy **Mercer’s condition** (must be positive semi-definite).
   - Example: String kernels for text, graph kernels for structured data.

### **Summary Table of Kernels in SVM (SVC & SVR)**

| **Kernel Type** | **Formula** | **Key Parameters** | **Best For** |
|----------------|------------|-------------------|-------------|
| **Linear** | $$K(x_i, x_j) = x_i^T x_j $$| None | Linear problems, high-dimensional data |
| **Polynomial** | $$ K(x_i, x_j) = (\gamma x_i^T x_j + r)^d $$ | `degree (d)`, `gamma (γ)`, `coef0 (r)` | Moderate non-linearity |
| **RBF (Gaussian)** | $$K(x_i, x_j) = \exp(-\gamma \|x_i - x_j\|^2) $$| `gamma (γ)` | Highly non-linear data (default choice) 
| **Sigmoid** | $$K(x_i, x_j) = \tanh (\gamma x_i^T x_j + r) $$| `gamma (γ)`, `coef0 (r)` | Neural network-like models |
| **Custom** | User-defined | Depends on implementation | Specialized problems |

### **Which Kernel to Choose?**
- **Linear Kernel**: Best for large feature spaces (e.g., text classification).
- **RBF Kernel**: Default choice for most non-linear problems.
- **Polynomial Kernel**: Useful when features have multiplicative interactions.
- **Sigmoid Kernel**: Rarely used, but can mimic neural networks.

In `scikit-learn`, these kernels can be used in `SVC` and `SVR` via the `kernel` parameter:
```python
from sklearn.svm import SVC, SVR

# Example: RBF Kernel for SVC
model = SVC(kernel='rbf', gamma=0.1)

# Example: Polynomial Kernel for SVR
model = SVR(kernel='poly', degree=3, gamma='scale')
```



### **Understanding Hyperparameters in SVM Kernels (SVC/SVR)**
When using kernels like `'linear'`, `'poly'` (polynomial), `'rbf'` (Radial Basis Function), and `'sigmoid'` in SVM, the choice of **hyperparameters** significantly impacts model performance. Below is a breakdown of key hyperparameters for each kernel and how they affect the decision boundary or regression fit.

---

## **1. Common Hyperparameters Across All Kernels**
These hyperparameters are shared by all kernels in `scikit-learn`'s `SVC` and `SVR`:

| Hyperparameter | Role | Default Value | Impact |
|--------------|------|--------------|--------|
| **`C`** | Regularization parameter | `1.0` | - **Small `C`**: More margin, allows misclassification (underfitting).<br>- **Large `C`**: Stricter margin, may overfit. |
| **`epsilon`** (SVR only) | Sensitivity to errors | `0.1` | - Larger `epsilon` → wider "tube" (more error tolerance).<br>- Smaller `epsilon` → stricter fit. |

---

## **2. Kernel-Specific Hyperparameters**
Each kernel has unique hyperparameters that control its flexibility.

### **A. Linear Kernel (`kernel='linear'`)**
- **No additional hyperparameters** (just `C`).
- **Decision boundary**: A straight line (or hyperplane in high dimensions).
- **Use case**: When data is (near) linearly separable.

### **B. Polynomial Kernel (`kernel='poly'`)**
| Hyperparameter | Role | Default | Impact |
|--------------|------|--------|--------|
| **`degree`** | Polynomial degree | `3` | - Higher `degree` → more complex curves (risk of overfitting). |
| **`gamma`** | Kernel coefficient | `'scale'` | - High `gamma` → sharper influence of each sample (risk of overfitting). |
| **`coef0`** | Independent term | `0.0` | - Controls bias (`r` in kernel formula). |

**Example:**
```python
from sklearn.svm import SVC
model = SVC(kernel='poly', degree=3, gamma='scale', coef0=1, C=1.0)
```

### **C. RBF (Gaussian) Kernel (`kernel='rbf'`)**
| Hyperparameter | Role | Default | Impact |
|--------------|------|--------|--------|
| **`gamma`** | Inverse kernel width | `'scale'` | - **Low `gamma`**: Smooth decision boundary (underfitting).<br>- **High `gamma`**: Tight fit around points (overfitting). |

**Example:**
```python
from sklearn.svm import SVR
model = SVR(kernel='rbf', gamma=0.1, C=10.0, epsilon=0.2)
```

### **D. Sigmoid Kernel (`kernel='sigmoid'`)**
| Hyperparameter | Role | Default | Impact |
|--------------|------|--------|--------|
| **`gamma`** | Scaling factor | `'scale'` | - Similar to RBF. |
| **`coef0`** | Bias term | `0.0` | - Shifts the activation threshold. |

**Example:**
```python
model = SVC(kernel='sigmoid', gamma=0.01, coef0=1, C=1.0)
```

---

## **3. How to Tune Hyperparameters?**
### **A. For `C` (Regularization)**
- Start with `C=1.0` and test `[0.01, 0.1, 1, 10, 100]`.
- **High `C`** → Low bias, high variance (overfitting).
- **Low `C`** → High bias, low variance (underfitting).

### **B. For `gamma` (RBF/Poly/Sigmoid)**
- `gamma='scale'` (default) uses `1 / (n_features * X.var())`.
- Try `[0.001, 0.01, 0.1, 1, 10]`.
- **High `gamma`** → Overfitting (tight fit).
- **Low `gamma`** → Underfitting (smoother fit).

### **C. For `degree` (Polynomial Kernel)**
- Start with `degree=2` or `3`.
- Higher degrees risk overfitting.

### **D. For `epsilon` (SVR)**
- Controls the width of the "insensitive" tube.
- Try `[0.01, 0.1, 0.5, 1.0]`.

---

## **4. Practical Recommendations**
| Kernel | Best For | Key Hyperparameters | Tuning Strategy |
|--------|---------|---------------------|-----------------|
| **Linear** | High-dimensional data | `C` | Adjust `C` for bias-variance tradeoff. |
| **Polynomial** | Moderate non-linearity | `degree`, `gamma`, `coef0` | Start with `degree=3`, tune `gamma`. |
| **RBF** | Default for non-linear | `gamma`, `C` | Use `GridSearchCV` on `gamma` and `C`. |
| **Sigmoid** | Rarely used | `gamma`, `coef0` | Similar to RBF but less stable. |

### **Example: Using `GridSearchCV` for RBF Kernel**
```python
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.01, 0.1, 1],
    'kernel': ['rbf']
}

grid = GridSearchCV(SVC(), param_grid, cv=5)
grid.fit(X_train, y_train)
print("Best params:", grid.best_params_)
```

---

## **5. Summary**
- **`C`**: Controls regularization (higher = stricter fit).
- **`gamma`**: Controls kernel width (higher = tighter fit).
- **`degree`**: Only for polynomial kernel (higher = more curves).
- **`coef0`**: Bias term in polynomial/sigmoid kernels.
- **`epsilon`**: Only for SVR (tolerance for errors).

For most cases, **RBF (`kernel='rbf'`)** with tuned `C` and `gamma` works best. Use `GridSearchCV` for optimal performance.

### **Correct Interpretation of `C` and `gamma` for Overfitting/Underfitting**  
Here’s a precise breakdown of how `C` and `gamma` control overfitting/underfitting in SVM kernels (RBF, Polynomial, etc.):

---

### **1. Regularization Parameter (`C`)**
- **Role**: Controls the trade-off between **maximizing margin width** and **minimizing classification errors**.  
- **Impact**:  
  - **High `C`** (e.g., `C=100`):  
    - The model **penalizes misclassifications more heavily**.  
    - Result: **Tighter decision boundary** (may overfit if `C` is too high).  
  - **Low `C`** (e.g., `C=0.01`):  
    - The model **allows more misclassifications** for a wider margin.  
    - Result: **Simpler (smoother) decision boundary** (may underfit if `C` is too low).  

#### **How to Adjust `C`?**
| Scenario           | Action       | Reason                                                                 |
|--------------------|--------------|------------------------------------------------------------------------|
| **Overfitting**    | Decrease `C` | Reduces strictness, allowing more errors for a broader margin.         |
| **Underfitting**   | Increase `C` | Makes the model stricter, fitting training data more closely.          |

---

### **2. Kernel Coefficient (`gamma`)**
- **Role**: Defines how far the influence of a single training example reaches (only for **RBF**, **Poly**, **Sigmoid** kernels).  
  - **Low `gamma`**:  
    - The kernel has a **wide influence** (smooth decision boundary).  
    - Similar to a **linear model** (may underfit).  
  - **High `gamma`**:  
    - The kernel has a **narrow influence** (tight fit around data points).  
    - May capture **noise** (overfitting).  

#### **How to Adjust `gamma`?**
| Scenario           | Action          | Reason                                                                 |
|--------------------|-----------------|------------------------------------------------------------------------|
| **Overfitting**    | Decrease `gamma`| Makes the kernel smoother, reducing sensitivity to noise.             |
| **Underfitting**   | Increase `gamma`| Allows the kernel to fit complex patterns (but risk overfitting).      |

---

### **3. Quick Reference Table**
| Hyperparameter | High Value → | Low Value → | Overfitting Fix | Underfitting Fix |
|---------------|-------------|-------------|-----------------|------------------|
| **`C`**      | Overfitting | Underfitting| **Decrease `C`** | **Increase `C`** |
| **`gamma`**  | Overfitting | Underfitting| **Decrease `gamma`** | **Increase `gamma`** |

---

### **4. Practical Example (RBF Kernel)**
#### **Case 1: Overfitting (High `C` + High `gamma`)**
- Symptoms:  
  - Training accuracy ≈ 100%, but validation accuracy is poor.  
  - Decision boundary is **too complex** (fits noise).  
- Fix:  
  ```python
  model = SVC(kernel='rbf', C=0.1, gamma=0.01)  # Reduce both C and gamma
  ```

#### **Case 2: Underfitting (Low `C` + Low `gamma`)**
- Symptoms:  
  - Poor performance on **both training and validation data**.  
  - Decision boundary is **too smooth** (misses patterns).  
- Fix:  
  ```python
  model = SVC(kernel='rbf', C=10, gamma=1.0)  # Increase both C and gamma
  ```

---

### **5. Pro Tips**
1. **Default Values**:  
   - `C=1.0`, `gamma='scale'` (auto-scales based on data variance).  
   - Start with defaults, then tune.  

2. **Use `GridSearchCV` for Automation**:  
   ```python
   from sklearn.model_selection import GridSearchCV
   param_grid = {
       'C': [0.1, 1, 10],
       'gamma': [0.01, 0.1, 1]
   }
   grid = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=5)
   grid.fit(X_train, y_train)
   print("Best params:", grid.best_params_)
   ```

3. **Visualize Decision Boundaries**:  
   - Use libraries like `mlxtend` to plot how `C` and `gamma` affect the model.  

---

### **Summary**
- **`C`** controls **how strictly misclassifications are penalized**.  
- **`gamma`** controls **how far the influence of a single point reaches**.  
- **Overfitting?** → Lower `C` and/or `gamma`.  
- **Underfitting?** → Increase `C` and/or `gamma`.  

This aligns with your intuition! Let me know if you'd like a deeper dive into any specific kernel.